In [1]:
import requests
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
import numpy as np
import random
from datetime import date
import time
import pandas as pd

""" Заполнили сведения из Headers при обращении к https://auto.ru/cars/used/ 
    (здесь указаны мои, полагаю, лучше использовать ваши)
    Не забудьте внести cookies"""
cookie = '_csrf_token=322d41e9006e0d84d35ea2ba160ffb0edcaf8bde6af076a0; suid=d5b39f92ec07e14b5ede65193afae8dd.b7a4c96080bd260714d8bcd31af9fb4a; autoruuid=g671fe5f02tuv4stgtm003ft6ufjgtuf.cf4d48bce1d92127b1dc5420a6c012d3; from=direct; autoru_sso_blocked=1; Session_id=3:1730143729.5.0.1729626845318:0EzFWw:1e52.1.2:1|1130000067438128.-1.0.3:1729626845|61:10026972.494636.G47KJoJnrGleXdU3AV1aAbBSOIE; sessar=1.1195.CiC4kiZ9xgGyA9EKYOSeDcPovH-CImZprpSTDFCpFXMUfQ.JxIZidV8vdomSfagoec4_A7vSfKJYsUeVx-hqv85aII; yandex_login=daboldyrev@edu.hse.ru; ys=udn.cDpkYWJvbGR5cmV2QGVkdS5oc2UucnU%3D#c_chck.1929241363; i=tlhDxtDyQeSf93G4Sd2jN8QeRk4YamYpZXEiLrhQS7CLpw0+5vhEzZB3fpv6+DoEc80kD0DF7VDIIgjCNuTIizOlOu8=; yandexuid=2160422051564510217; my=YwA=; L=Xy5fSWVnBHIJSEN2UVlaRQ5hWW9bXVoGIFYzOSssLABdEC9SECxcJxsSVytC.1729626845.15927.340609.84c416984beb9e0cf870a4d6f92acb61; mda2_beacon=1730143729098; sso_status=sso.passport.yandex.ru:synchronized; autoru_sso_redirect_blocked=1; _ym_uid=1684353637548378497; yaPassportTryAutologin=1; _ym_isad=2; autoru_sid=123729954%7C1730143739148.7776000.b4c5nf1yViUEOZ-QftLYSw.mUjU3EqWVT1cPoNPBDcE6l2ZGA5w6n6ifxDp-_Y12Pc; _yasc=+J+VIii3pr3WbEOhoe+sjg44LQZA886gpGdnqTZiAIEC+O3+kkAC+EqS5keXehajStIA; popups-popup-guess-the-car-shown-count=1; _ym_d=1730145169; count-visits=4; from_lifetime=1730145175566; layout-config={"screen_height":1080,"screen_width":1920,"win_width":334,"win_height":983}; cycada=Crddb5c+sUwEnCqwO6jVWnYzs+eP5+y9mEaD7l6eSdg='
headers = {
    'authority': 'auto.ru',
    'method': 'GET',
    'path': '/cars/used/',
    'scheme': 'https',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'accept-encoding': 'gzip, deflate, br, zstd',
    'accept-language': 'ru,en;q=0.9,en-GB;q=0.8,en-US;q=0.7',
    'cache-control': 'max-age=0',
    'cookie': cookie,
    'priority': 'u=0, i',
    'sec-ch-ua': '"Chromium";v="130", "Microsoft Edge";v="130", "Not?A_Brand";v="99"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform':'"Windows"',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36 Edg/130.0.0.0'
}

In [2]:
def get_car(link):   
    
    """Функция собирает сведения со странички объявления"""

    url_car = link # записали ссылку

    # загрузили страницу с данными по автомобилю
    page_auto = requests.get(link, headers=headers)
        
    # создали дерево
    soup_auto = BeautifulSoup(page_auto.content, 'html.parser')

    # извлекли данные из блока CardBreadcrumbs
    CardBreadcrums = soup_auto.find('div', {'class': 'CardBreadcrumbs'})
    car_make = CardBreadcrums.find_all('a')[2].text
    car_model = CardBreadcrums.find_all('a')[3].text
    car_gen = CardBreadcrums.find_all('a')[4].text
    car_type = CardBreadcrums.find_all('a')[5].text
    car_compl = CardBreadcrums.find_all('a')[6].text

    # извлекли данные об объявлении
    ann_date = soup_auto.find('div', {'class': 'CardHead__infoItem CardHead__creationDate'}).text
    ann_id = soup_auto.find('div', {'class': 'CardHead__infoItem CardHead__id'}).text
    car_price = soup_auto.find('span', {'class': 'OfferPriceCaption__price'}).text.replace('\xa0','')
    ann_city = soup_auto.find('span', {'class': 'MetroListPlace__regionName MetroListPlace_nbsp'}).text
        
    # Нашли нужный блок с информацией, ищем только по нему, бережем ресурсы
    card_info = soup_auto.find('div', {'class': 'CardInfo-ateuv'})

    # записали ссылку на комплектацию
    link_cpl = card_info.find('a').get('href')
        
    # извлекли сведения об автомобиле
    avail = card_info.find('li', {'class': 'CardInfoRow CardInfoRow_availability'}).find_all('div')[1].text
    year = card_info.find('li', {'class': 'CardInfoRow CardInfoRow_year'}).find_all('div')[1].text
    mileage = card_info.find('li', {'class': 'CardInfoRow CardInfoRow_kmAge'}).find_all('div')[1].text
    color = card_info.find('li', {'class': 'CardInfoRow CardInfoRow_color'}).find_all('div')[1].text
    if card_info.find('li', {'class': 'CardInfoRow CardInfoRow_engine'}).find_all('div')[1].text.split(' / ')[2] == 'Электро':
        eng_power, eng_power_kw, eng_type = card_info.find('li', {'class': 'CardInfoRow CardInfoRow_engine'}).find_all('div')[1].text.split(' / ')
        eng_power_kw = int(eng_power_kw.replace('\xa0',' ').split()[0])
        eng_size = None
        pow_resrv = card_info.find('li', {'class': 'CardInfoRow CardInfoRow_electricRange'}).find_all('div')[1].text.split()[0]
    else:
        eng_size, eng_power, eng_type = card_info.find('li', {'class': 'CardInfoRow CardInfoRow_engine'}).find_all('div')[1].text.split(' / ')
        eng_size = float(eng_size.split()[0])
        eng_power_kw = None
        pow_resrv = None
    options = card_info.find('li', {'class': 'CardInfoRow CardInfoRow_complectationOrEquipmentCount'}).find_all('div')[1].text
    transmission = card_info.find('li', {'class': 'CardInfoRow CardInfoRow_transmission'}).find_all('div')[1].text
    drive = card_info.find('li', {'class': 'CardInfoRow CardInfoRow_drive'}).find_all('div')[1].text
    st_wheel= card_info.find('li', {'class': 'CardInfoRow CardInfoRow_wheel'}).find_all('div')[1].text
    condition = card_info.find('li', {'class': 'CardInfoRow CardInfoRow_state'}).find_all('div')[1].text
    count_owner = card_info.find('li', {'class': 'CardInfoRow CardInfoRow_ownersCount'}).find_all('div')[1].text
    try:
        original_pts = card_info.find('li', {'class': 'CardInfoRow CardInfoRow_pts'}).find_all('div')[1].text
    except:
        original_pts = None
    try:
        customs = card_info.find('li', {'class': 'CardInfoRow CardInfoRow_customs'}).find_all('div')[1].text
    except:
        customs = None

    # подготовили словарь для месяцев
    dict_month = {
        'января': 1,
        'февраля': 2,
        'марта': 3,
        'апреля': 4,
        'мая': 5,
        'июня': 6,
        'июля': 7,
        'августа': 8,
        'сентября': 9,
        'октября': 10,
        'ноября': 11,
        'декабря':12
    }

    # записали данные в словарь
    row = {
        'url_car': url_car,
        'car_make': car_make,
        'car_model': car_model,
        'car_gen': car_gen,
        'car_type': car_type,
        'car_compl': car_compl,
        'ann_date': str(date(2024, dict_month[ann_date.split()[1]], int(ann_date.split()[0]))),
        'ann_id': ann_id[2:],
        'car_price': int(car_price[:-1]),
        'ann_city': ann_city,
        'link_cpl': link_cpl,
        'avail': avail.replace('\xa0',' '),
        'year': int(year),
        'mileage': int(mileage.replace('\xa0','')[:-3]),
        'color': color,
        'eng_size': eng_size,
        'eng_power': int(eng_power.replace('\xa0',' ').split()[0]),
        'eng_power_kw': eng_power_kw,
        'eng_type': eng_type,
        'pow_resrv': pow_resrv,
        'options': options.replace('\xa0',' '),
        'transmission': transmission,
        'drive': drive,
        'st_wheel': st_wheel,
        'condition': condition,
        'count_owner': count_owner.replace('\xa0',' '),
        'original_pts': original_pts,
        'customs': customs
        }
    
    # передаем словарь
    return row


In [3]:
def get_page(filtered_url, p):

    """ Функция собирает ссылки на авто со странички 
        и запускает функцию сбора информации по каждому найденному объявлению"""

    # изготовили ссылку на страницу списка с объявлениями
    if p != 1:
        url = f'{filtered_url}&page={p}'
    else:
        url = filtered_url

    # сходили по ней
    page_list = requests.get(url, headers=headers)

    # построили дерево
    soup = BeautifulSoup(page_list.content, 'html.parser')

    # нашли в нём ссылки на авто, записали в список
    urls = []
    for link in soup.find_all('a'):
        if link.get('href') != None and 'cars/used/sale' in link.get('href')  and link.get('href') not in urls:
            urls.append(link.get('href'))

    # прошли по ссылкам записали данные в список словарей
    info = []
    mistakes = []
    random_seconds = [3, 5, 7, 10, 12, 15]
    for link in tqdm(urls):
        time.sleep(random.choice(random_seconds)) # немного подождали
        try:
            info.append(get_car(link))
        except:
            print(f'Ошибка при загрузке страницы: {link}')
            mistakes.append(link)

    return info

In [4]:
pages_to_scrap = pd.read_excel('pages_to_scrap.xlsx', engine='openpyxl')
pages_to_scrap.head()

,mark,filter,amount,ref,suffix
0,Lada (ВАЗ),"1111 Ока, 2101, 2102, 2103, 2104, 2105",3116,https://auto.ru/cars/used/?catalog_filter=mark...,&page=
1,Lada (ВАЗ),"2106, 2108, 2109, 2111",3628,https://auto.ru/cars/used/?catalog_filter=mark...,&page=
2,Lada (ВАЗ),"2107, 2120 Надежда, 2123, 2329",3534,https://auto.ru/cars/used/?catalog_filter=mark...,&page=
3,Lada (ВАЗ),"21099, 2112",3562,https://auto.ru/cars/used/?catalog_filter=mark...,&page=
4,Lada (ВАЗ),"2110, 2113",3580,https://auto.ru/cars/used/?catalog_filter=mark...,&page=


In [ ]:
for i  in tqdm(range(0, pages_to_scrap.shape[0])):
    url_to_scrap = pages_to_scrap['ref'][i]
    max_page = int(pages_to_scrap['amount'][i] // 37 + 1)
    # записали сведения в лог
    with open('log.txt', 'a') as log:
        print(f'Фильтр {url_to_scrap}\nстраниц {max_page}, записей {pages_to_scrap['amount'][i]}', file=log)    
    for page in tqdm(range(1, max_page)):
        try:
            date_page = get_page(url_to_scrap, page)
            # дописали данные в файл
            with open('data.txt', 'a') as data:
                for record in date_page:
                    print(record, file=data)
            # записали сведения в лог
            with open('log.txt', 'a') as log:
                print(f'Успешно загружена страница {page}. Число добавленных записей {len(date_page)}',
                file=log)
        except:
            # записали сведения в лог
            with open('log.txt', 'a') as log:
                print(f'Не была загружена страница {page}. Число добавленных записей {0}', file=log)
        
        # закрыли файлы
        data.close()
    log.close()

  0%|          | 0/39 [00:00<?, ?it/s]

  0%|          | 0/84 [00:00<?, ?it/s]

  0%|          | 0/37 [00:00<?, ?it/s]

  0%|          | 0/38 [00:00<?, ?it/s]

Ошибка при загрузке страницы: https://auto.ru/cars/used/sale/vaz/2104/1125548201-d793bff9/


...

  0%|          | 0/38 [00:00<?, ?it/s]

Ошибка при загрузке страницы: https://auto.ru/cars/used/sale/vaz/2112/1124966289-c987b39a/


  0%|          | 0/37 [00:00<?, ?it/s]

  0%|          | 0/38 [00:00<?, ?it/s]

Ошибка при загрузке страницы: https://auto.ru/cars/used/sale/vaz/21099/1124519473-fb5771ba/
Ошибка при загрузке страницы: https://auto.ru/cars/used/sale/vaz/2112/1125196263-f08ac64a/
Ошибка при загрузке страницы: https://auto.ru/cars/used/sale/vaz/2112/1124086594-e49e6353/


In [7]:
# Смотрим на пример того, что приходит
date_page[1]

{'url_car': 'https://auto.ru/cars/used/sale/ford/focus/1124411293-618720a3/',
 'car_make': 'Ford',
 'car_model': 'Focus',
 'car_gen': 'II Рестайлинг',
 'car_type': 'Седан',
 'car_compl': '1.8 MT (125 л.с.)',
 'ann_date': '2024-08-03',
 'ann_id': '1124411293',
 'car_price': 1050000,
 'ann_city': 'Краснодар',
 'link_cpl': 'https://auto.ru/catalog/cars/ford/focus/2306579/2306701/specifications/2306701_20192891_2306706/',
 'avail': 'В наличии',
 'year': 2010,
 'mileage': 162456,
 'color': 'серый',
 'eng_size': 1.8,
 'eng_power': 125,
 'eng_power_kw': None,
 'eng_type': 'Бензин',
 'pow_resrv': None,
 'options': 'Titanium',
 'transmission': 'механическая',
 'drive': 'передний',
 'st_wheel': 'Левый',
 'condition': 'Не требует ремонта',
 'count_owner': '2 владельца',
 'original_pts': 'Оригинал',
 'customs': 'Растаможен'}